In [ ]:
!pip install acnportal

In [ ]:
from acnportal import acndata
from datetime import datetime
import pytz
import pandas as pd
import numpy as np
import json
from google.colab import files
from collections import Counter
import numpy as np

In [ ]:
# Inputs
timezone = pytz.timezone("US/Pacific")
start = timezone.localize(datetime(2018, 12, 14)) #used for paper
end = timezone.localize(datetime(2018, 12 , 15))
# start = timezone.localize(datetime(2019, 3, 1))
# end = timezone.localize(datetime(2019, 3 , 2))
site = "jpl"
#site = "jpl"
# site = "office01"

In [ ]:
client = acndata.DataClient("Huc5u86zT6zhlOJjo0tjgRRSg-yIKeFBtV3sIfIx0LM")

In [ ]:
sessions = client.get_sessions_by_time(site, start, end, timeseries=True)
df = pd.DataFrame(sessions)

In [ ]:
cleaned_df = df.dropna()
cleaned_df['userID'] = cleaned_df['userID'].astype(int)
cleaned_df.reset_index(drop=True, inplace=True)
cleaned_df

In [ ]:
userIDS=[169,365,368,374,405,406,410,474,483,531,576,592,607,651]

In [ ]:
# Assuming cleaned_df has already been cleaned for None values as discussed, and now we need to filter it for specific userIDs

# List of userIDs to filter
userIDs = [169, 365, 368, 374, 405, 406, 410, 474, 483, 531, 576, 592, 607, 651]

# Filter the DataFrame for the specified userIDs
filtered_cleaned_df = cleaned_df[cleaned_df['userID'].isin(userIDs)]

# Keep only the first instance of each userID
filtered_cleaned_df = filtered_cleaned_df.drop_duplicates(subset='userID', keep='first')

# Reset the index for clarity
filtered_cleaned_df.reset_index(drop=True, inplace=True)

filtered_cleaned_df

In [ ]:
import pytz
from datetime import datetime

# Loop through each row in the DataFrame
for i in range(len(filtered_cleaned_df)):
    element = filtered_cleaned_df.iloc[i, 1]

    # Parse the departure datetime from the string
    # requested_departure_str = element[0]['requestedDeparture']
    # requested_departure = datetime.strptime(requested_departure_str, '%a, %d %b %Y %H:%M:%S GMT')
    # requested_departure = timezone.localize(requested_departure)  # Localize if your datetime strings are in GMT
    # start=element[0]['modifiedAt']
    # start = datetime.strptime(start, '%a, %d %b %Y %H:%M:%S GMT')
    # start = timezone.localize(start)  # Localize if your datetime strings are in GMT
    # Calculate the number of 5-minute intervals since the start date
    intervals=np.ceil(element[0]['minutesAvailable']/5)

    # Print results
    print(f"User ID: {element[0]['userID']}, SessionID: {filtered_cleaned_df.iloc[i, 2]}, requested departure: {intervals} intervals, kWh requested: {element[0]['kWhRequested']}, capacity:{filtered_cleaned_df.iloc[i, 9]}")


In [ ]:
# `threshold=np.inf` means print all elements, no truncation.
np.set_printoptions(threshold=np.inf)


In [ ]:
print(x['userInputs'])

In [ ]:
import pandas as pd
import gzip

# Assuming 'df' is your DataFrame
json_str = filtered_cleaned_df.to_json(orient='records', lines=True)

# Compress the JSON string and save as .gz file
with gzip.open('/content/sample_data/mydata.json.gz', 'wt', encoding='utf-8') as f:
    f.write(json_str)


In [ ]:
df_filtered = df.drop(['_id','userInputs','sessionID','stationID','spaceID','siteID','clusterID','pilotSignal','chargingCurrent','doneChargingTime','timezone'], axis=1)

In [ ]:
import pandas as pd

# Convert the columns to string type, if they aren't already
df_filtered['connectionTime'] = df_filtered['connectionTime'].astype(str)
df_filtered['disconnectTime'] = df_filtered['disconnectTime'].astype(str)

# Remove the -08:00 from both columns
df_filtered['connectionTime'] = df_filtered['connectionTime'].str.replace(r'-\d{2}:\d{2}$', '')
df_filtered['disconnectTime'] = df_filtered['disconnectTime'].str.replace(r'-\d{2}:\d{2}$', '')

# Convert the columns to datetime format
df_filtered['connectionTime'] = pd.to_datetime(df_filtered['connectionTime'])
df_filtered['disconnectTime'] = pd.to_datetime(df_filtered['disconnectTime'])

# Subtract the columns to get the difference in hours
df_filtered['duration'] = (df_filtered['disconnectTime'] - df_filtered['connectionTime']).dt.total_seconds() / 3600



In [ ]:
df_filtered

In [ ]:
df_filtered_2=df_filtered[[ 'connectionTime', 'disconnectTime', 'kWhDelivered','userID','duration']]
df_filtered_2

In [ ]:
data_tuples = df_filtered_2.apply(tuple, axis=1).tolist()
data_tuples_vector = np.array(data_tuples)

In [ ]:
# Filter out tuples where the first element is None
data_tuples_vector = np.array([t for t in data_tuples_vector if t[3] is not None])

# Sort according to input ID
sorted_indices = np.argsort([t[3] for t in data_tuples_vector])
sorted_data_tuples_vector = data_tuples_vector[sorted_indices]

In [ ]:
sorted_data_tuples_vector

In [ ]:
# Extract the first elements (ignoring None) from sorted_column_vector
first_elements = [t[3] for t in sorted_data_tuples_vector if t[3] is not None]

# Count occurrences of each unique value
counts = Counter(first_elements)

# Add the counts as a new column to sorted_column_vector
extended_vector = np.column_stack((sorted_data_tuples_vector, [counts[t[3]] if t[3] is not None else 0 for t in sorted_data_tuples_vector]))

# Sort based on the new column (count column) in descending order
sorted_indices = np.argsort(-extended_vector[:,5].astype(int))
final_sorted_vector = extended_vector[sorted_indices]

print(final_sorted_vector)

In [ ]:
df_sorted_data_tuples=pd.DataFrame(final_sorted_vector)

In [ ]:
df_sorted_data_tuples

In [ ]:
df_sorted_data_tuples.columns = ['connectionTime','disconnectTime','kWhDelivered','userID', 'duration','no_sessions']

In [ ]:
df_sorted_data_tuples

In [ ]:
no_sessions_threshold = 20  # change to desired value

# Filter rows where the last column is greater than or equal to the threshold
df_filtered_sorted_data_tuples= df_sorted_data_tuples[df_sorted_data_tuples.iloc[:, -1] >= no_sessions_threshold]

print(df_filtered_sorted_data_tuples)

In [ ]:
df_filtered_sorted_data_tuples.to_csv("caltech_training_data_3_years.csv")
files.download('caltech_training_data_3_years.csv')

Additional code that creates an array with just the user ID and number of charging sessions

In [ ]:
from collections import Counter
import numpy as np

#Create array that includes user ID and number of
#charging sessions
first_elements = [t[0] for t in sorted_data_tuples_vector if t[0] is not None]  # Extracting first elements and ignoring None
counts = Counter(first_elements)

sorted_counts = sorted(counts.items(), key=lambda x: x[0])
ID_and_charge = np.array(sorted_counts)

#Sort users in decreasing order according to number of
#charging sessions
# Create a separate integer array for the counts
counts_int = ID_and_charge[:, 1].astype(int)

# Get indices that would sort the counts in decreasing order
sorted_indices = np.argsort(counts_int)[::-1]

# Reorder the result_array using these indices
sorted_ID_and_charge = ID_and_charge[sorted_indices]


In [ ]:
sorted_ID_and_charge